In [1]:
import pandas as pd
flights = pd.read_csv("/content/drive/MyDrive/flights.csv")
flights.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [2]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = flights

In [4]:
# Drop the following columns based on theory, not the data
# All data are from the same year
df = df.drop(['YEAR'], axis=1)

# Cancelled flights never happened
df = df.drop(['CANCELLED', 'CANCELLATION_REASON'], axis=1)

# Actual flying data, couldn't know beforehand
df = df.drop(['DEPARTURE_TIME', 'DEPARTURE_DELAY',
              'TAXI_OUT', 'WHEELS_OFF', 'ELAPSED_TIME',
              'AIR_TIME', 'WHEELS_ON', 'TAXI_IN', 'ARRIVAL_TIME'], axis=1)

In [5]:
# Drop all rows where ARRIVAL_DELAY is missing, we need a y variable for prediction
df = df.dropna(subset=['ARRIVAL_DELAY'])

In [6]:
# Select only the top 20 busiest airports, based on https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States
airport_list = ['ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'SEA', 'LAS', 'MCO',
                'CLT', 'EWR', 'PHX', 'IAH', 'MIA', 'BOS', 'MSP', 'DTW', 'FLL', 'PHL']

DEST_check = df.DESTINATION_AIRPORT.isin(airport_list)
df = df[DEST_check]

ORIG_check = df.ORIGIN_AIRPORT.isin(airport_list)
df = df[ORIG_check]

In [7]:
# Check N/A%
df.isna().sum()/len(flights)*100

MONTH                   0.00000
DAY                     0.00000
DAY_OF_WEEK             0.00000
AIRLINE                 0.00000
FLIGHT_NUMBER           0.00000
TAIL_NUMBER             0.00000
ORIGIN_AIRPORT          0.00000
DESTINATION_AIRPORT     0.00000
SCHEDULED_DEPARTURE     0.00000
SCHEDULED_TIME          0.00000
DISTANCE                0.00000
SCHEDULED_ARRIVAL       0.00000
ARRIVAL_DELAY           0.00000
DIVERTED                0.00000
AIR_SYSTEM_DELAY       17.37146
SECURITY_DELAY         17.37146
AIRLINE_DELAY          17.37146
LATE_AIRCRAFT_DELAY    17.37146
WEATHER_DELAY          17.37146
dtype: float64

In [8]:
# Drop the following bc of high N/A%
df = df.drop(['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
              'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis=1)

In [9]:
# Check for the num of unique values in each column
df.nunique()

MONTH                    11
DAY                      31
DAY_OF_WEEK               7
AIRLINE                  13
FLIGHT_NUMBER          5715
TAIL_NUMBER            4563
ORIGIN_AIRPORT           20
DESTINATION_AIRPORT      20
SCHEDULED_DEPARTURE    1231
SCHEDULED_TIME          374
DISTANCE                182
SCHEDULED_ARRIVAL      1387
ARRIVAL_DELAY           918
DIVERTED                  1
dtype: int64

In [10]:
# Drop FLIGHT_NUMBER and TAIL_NUMBER bc of too many unique values and no way of grouping
df = df.drop(['FLIGHT_NUMBER', 'TAIL_NUMBER'], axis=1)

# Drop DIVERTED bc it only has 1 unique value
df = df.drop(['DIVERTED'], axis=1)

In [11]:
# Transform SCHED_DEPARTURE and SCHED_ARRIVAL into 4 quarters of a day, with an interval of 6 hours
import numpy as np
df['SCHED_DEPARTURE'] = np.ceil(df['SCHEDULED_DEPARTURE']/600).apply(int)
df['SCHED_ARRIVAL'] = np.ceil(df['SCHEDULED_ARRIVAL']/600).apply(int)
# Drop the original columns
df = df.drop(['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL'], axis=1)

In [12]:
# Create dummies for categorical data
AIRLINE_dummies = pd.get_dummies(df['AIRLINE'], prefix='AIRLINE', drop_first=True)
ORIG_dummies = pd.get_dummies(df['ORIGIN_AIRPORT'], prefix='ORIG', drop_first=True)
DEST_dummies = pd.get_dummies(df['DESTINATION_AIRPORT'], prefix='DEST', drop_first=True)

df = df.drop(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], axis=1)
df = pd.concat([df, AIRLINE_dummies, ORIG_dummies, DEST_dummies], axis=1)

In [13]:
# Transform ARRIVAL_DELAY into 3 categories, making it a classification problem
DELAY_STATUS =[]
for value in df['ARRIVAL_DELAY']:
    if value > 120:
        DELAY_STATUS.append(3)
    elif value > 60:
        DELAY_STATUS.append(2)
    elif value > 30:
        DELAY_STATUS.append(1)
    else:
        DELAY_STATUS.append(0)  

df['DELAY_STATUS'] = DELAY_STATUS
# Drop the original column
df = df.drop('ARRIVAL_DELAY', axis=1)

In [14]:
# Check the distribution of the categories
df.DELAY_STATUS.value_counts()

0    1116489
1      78774
2      51873
3      29258
Name: DELAY_STATUS, dtype: int64

In [15]:
# Finally, here is our cleaned data
df.head()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_TIME,DISTANCE,SCHED_DEPARTURE,SCHED_ARRIVAL,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,...,DEST_LAX,DEST_MCO,DEST_MIA,DEST_MSP,DEST_ORD,DEST_PHL,DEST_PHX,DEST_SEA,DEST_SFO,DELAY_STATUS
2,1,1,4,286.0,2296,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,4,285.0,2342,1,2,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,1,1,4,217.0,1589,1,2,0,0,1,...,0,0,0,1,0,0,0,0,0,0
6,1,1,4,181.0,1299,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,1,1,4,273.0,2125,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Standardize and train/test split
data = df.values

from sklearn.preprocessing import StandardScaler
X_og = data[:,:-1]
sc = StandardScaler()
X = sc.fit_transform(X_og)
y = data[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [17]:
# Base tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', max_depth=5)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict_proba(X_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred, multi_class='ovr')

0.6338334790927004

In [18]:
# Bagged Tree
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), n_estimators=20)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8743702976347478

In [19]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 20, criterion = 'gini', random_state = 42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.8621614492882307

In [20]:
# The run time for XGBoost is too long, therefore, we did not let it finish running
# # XGBoost
# import xgboost as xgb
# clf = xgb.XGBClassifier()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

# accuracy_score(y_test, y_pred)

In [21]:
# lightGBM
import lightgbm as lgbm
clf = lgbm.LGBMClassifier(objective = "binary")
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)

roc_auc_score(y_test, y_pred, multi_class= "ovr")

0.7256735334025103